# Computer Exercise 2
#### Due date: Oct. 5 (Wed.) 23:00-2022312873李佳璇

In [1]:
# import libraries
import csv
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Experiment 1 (FLD):
#### Use TrainingSet-1 to calculate the discriminant function using FLD. Apply the discriminant function on TestSet-1. Calculate the error rate. 
### Step 1: Import training data 
#### Include data in 'train1_icu_data.csv' as trainX, and 'train1_icu_label.csv' as tarinY

In [2]:
# load trainX
trainX = []
print('For feature x:')
with open('./data1forEx1to4/train1_icu_data.csv') as xf:
    xf_csv = csv.reader(xf)
    # get headers
    headersX = next(xf_csv)
    print('-------------------------------feature counting-------------------------------------')
    print('Totally %d features' % len(headersX),'included:\n', headersX)
    # get x values
    for xrow in xf_csv:
        trainX.append(xrow)
    print('-------------------------------sample counting-------------------------------------')
    print('There are totally %d samples for training' % len(trainX))

For feature x:
-------------------------------feature counting-------------------------------------
Totally 108 features included:
 ['age', 'bmi', 'elective_surgery', 'height', 'pre_icu_los_days', 'weight', 'apache_2_diagnosis', 'apache_3j_diagnosis', 'arf_apache', 'bun_apache', 'creatinine_apache', 'gcs_eyes_apache', 'gcs_motor_apache', 'gcs_verbal_apache', 'glucose_apache', 'heart_rate_apache', 'hematocrit_apache', 'intubated_apache', 'map_apache', 'resprate_apache', 'sodium_apache', 'temp_apache', 'ventilated_apache', 'wbc_apache', 'd1_diasbp_max', 'd1_diasbp_min', 'd1_heartrate_max', 'd1_heartrate_min', 'd1_mbp_max', 'd1_mbp_min', 'd1_resprate_max', 'd1_resprate_min', 'd1_spo2_max', 'd1_spo2_min', 'd1_sysbp_max', 'd1_sysbp_min', 'd1_temp_max', 'd1_temp_min', 'h1_diasbp_max', 'h1_diasbp_min', 'h1_heartrate_max', 'h1_heartrate_min', 'h1_mbp_max', 'h1_mbp_min', 'h1_resprate_max', 'h1_resprate_min', 'h1_spo2_max', 'h1_spo2_min', 'h1_sysbp_max', 'h1_sysbp_min', 'h1_temp_max', 'h1_temp_m

In [3]:
# load trainY
trainY = []
print('For label y:')
with open('./data1forEx1to4/train1_icu_label.csv') as yf:
    yf_csv = csv.reader(yf)
    # get headers
    headersY= next(yf_csv)
    print('-------------------------------label counting-------------------------------------')
    print('Totally %d label' % len(headersY),'included:\n', headersY)
    for yrow in yf_csv:
        trainY.append(yrow)
    print('-------------------------------sample counting-------------------------------------')
    if len(trainY)==len(trainX):
        print('There are totally %d samples for training' % len(trainY), '，which has the same counting result as X')
    else:
        print('Index error: X and Y have different sample size.')

For label y:
-------------------------------label counting-------------------------------------
Totally 1 label included:
 ['hospital_death']
-------------------------------sample counting-------------------------------------
There are totally 5000 samples for training ，which has the same counting result as X


### (Extra) Normalize the data
#### As we can see that the number scope of 108 features have large difference, do a normalization here to make them have the same order of magnitude, which is good for comprehensive comparison.

In [4]:
# 0-1 normalization
AtrainX = np.array(trainX, dtype = 'float64')
def maxminNorm(arrX):
    maxi = np.max(arrX, axis=0)
    mini = np.min(arrX, axis=0)
    return (arrX-mini)/(maxi-mini)
AtrainX = maxminNorm(AtrainX)

### Step 2: Divide the feature X into two groups according to label Y
#### label Y indicate the hospital_death, wich is the classification. So using the label difference to divide X into two groups. One group of people with feature X0 with hospital_death=0, and the other group of people with feature X1 with hospital_death=1.

In [5]:
AtrainY = np.array(trainY, dtype = 'float64')
AtrainY = AtrainY.reshape(len(trainY),)
# make sure the number of 0,1 is not a string but a integer
for i in range(len(trainY)):
    AtrainY[i]=int(AtrainY[i])
# division
pos0 = np.where(AtrainY==0)
AXneg = np.squeeze(AtrainX[pos0,:])
pos1 = np.where(AtrainY==1)
AXpos = np.squeeze(AtrainX[pos1,:])
print('After division,')
print('We have a group of hospital_death(Ylabel)=0 with %d samples' %AXneg.shape[0])
print('We have a group of hospital_death(Ylabel)=1 with %d samples' %AXpos.shape[0])

After division,
We have a group of hospital_death(Ylabel)=0 with 2450 samples
We have a group of hospital_death(Ylabel)=1 with 2550 samples


### Step 3: Matrix calculation
#### Calculate within-class scatter matrix and Between-class scatter matrix for further FLD calculation

In [7]:
# calculate class mean
mean0 = np.mean(AXneg, axis=0)
mean1 = np.mean(AXpos, axis=0)
Amean0 = mean0.reshape(mean0.shape[0],1)
Amean1 = mean1.reshape(mean1.shape[0],1)
# within-class scatter matrix, S1 for class 0, S2 for class 1
S1 = np.dot((AXneg-mean0).transpose(), (AXneg-mean0))
S2 = np.dot((AXpos-mean1).transpose(), (AXpos-mean1))
Sw = S1 + S2  # (108, 108)
# between-class scatter matrix

Sb = np.dot((Amean0-Amean1), (Amean0-Amean1).transpose())  # (108, 108)

# check whether Sw is invertible
if np.linalg.det(Sw)!=0:
    print('Sw is invertible')
else:
    print('Warning!!!Sw is noninvertible')

Sw is invertible


### Step 4: Find the eigenvalue and eigenvector
#### find 𝒘∗ as the eigenvector of matrix inverse(Sw)Sb

In [334]:
W = np.dot(np.linalg.inv(Sw),(Amean0-Amean1))
Wabs = np.linalg.norm(W, axis=1, keepdims=True)
W = W/Wabs
print(W.shape)


(108, 1)


In [336]:

# cov = np.cov(AXneg, rowvar=False) + np.cov(AXpos, rowvar=False)
# w = np.linalg.inv(cov) @ (Amean0 - Amean1)
# w = w/np.linalg.norm(w, axis=1, keepdims=True)


aa = AXneg[54].reshape(108,1)
w0 = -1/5000*(2550)
print(w0)

print(np.dot(w.transpose(), aa))
print(np.matrix(AXneg).dot(W)+w0)
print(np.matrix(AXpos).dot(W)+w0)




-0.51
[[-766.45421053]]
[[-587.0437345 ]
 [-576.70264626]
 [ -66.86060262]
 ...
 [-673.01764214]
 [-589.3729833 ]
 [-324.22647498]]
[[ -828.70691043]
 [ -357.98820294]
 [ -856.62932761]
 ...
 [-1093.63584266]
 [ -602.26795703]
 [-1018.17608377]]


## Experiment 2 (Perceptron):
#### 1) Use TrainingSet-1 to train the Perceptron classifier. Calculate the training error and cross validation error on the training set. Apply the trained Perceptron to TestSet-1. Compare the training error rate, cross-validation error rate and test error rate.
#### 2) Use TrainingSet-2 to train the Perceptron classifier. Apply the Perceptron trained with TrainingSet-2 to TestSet-1 and TestSet-2, respectively. Compare the error rates.
#### 3) Discuss your observations on the results.

### Step 1: Import training data 
#### Include data in 'train1_icu_data.csv' as trainX, and 'train1_icu_label.csv' as tarinY